In [30]:
import praw
from SPY_data import *
import time
import pandas as pd
import matplotlib.pyplot as plt
import squarify
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import plotly.express as px

In [2]:
start_time = time.time()
reddit = praw.Reddit(user_agent="Comment Extraction",
    client_id="FKWnWESEV0BOiw",
    client_secret="KEsgePh4VAr6wlfXhG4L5T_kS3zO1A",
    username="Zz0z77",
    password="D3l1=oyLprA",
    check_for_async=False)

In [3]:
# set the program parameters
subs = ['wallstreetbets', 'stocks', 'investing', 'stockmarket']     # sub-reddit to search
post_flairs = {'Daily Discussion', 'Weekend Discussion', 'Discussion'}    # posts flairs to search || None flair is automatically considered
goodAuth = {'AutoModerator'}   # authors whom comments are allowed more than once
uniqueCmt = True                # allow one comment per author per symbol
ignoreAuthP = {'example'}       # authors to ignore for posts 
ignoreAuthC = {'example'}       # authors to ignore for comment 
upvoteRatio = 0.70         # upvote ratio for post to be considered, 0.70 = 70%
ups = 20       # define # of upvotes, post is considered if upvotes exceed this #
limit = 100      # define the limit, comments 'replace more' limit
upvotes = 2     # define # of upvotes, comment is considered if upvotes exceed this #
picks = 10     # define # of picks here, prints as "Top ## picks are:"
picks_ayz = 5   # define # of picks for sentiment analysis

In [4]:
posts, count, c_analyzed, tickers, titles, a_comments = 0, 0, 0, {}, [], {}
cmt_auth = {}

for sub in subs:
    subreddit = reddit.subreddit(sub)
    hot_python = subreddit.hot()    # sorting posts by hot
    # Extracting comments, symbols from subreddit
    for submission in hot_python:
        flair = submission.link_flair_text 
        author = submission.author.name         
        
        # checking: post upvote ratio # of upvotes, post flair, and author 
        if submission.upvote_ratio >= upvoteRatio and submission.ups > ups and (flair in post_flairs or flair is None) and author not in ignoreAuthP:   
            submission.comment_sort = 'new'     
            comments = submission.comments
            titles.append(submission.title)
            posts += 1
            submission.comments.replace_more(limit=limit)   
            for comment in comments:
                # try except for deleted account?
                try: auth = comment.author.name
                except: pass
                c_analyzed += 1
                
                # checking: comment upvotes and author
                if comment.score > upvotes and auth not in ignoreAuthC:      
                    split = comment.body.split(" ")
                    for word in split:
                        word = word.replace("$", "")        
                        # upper = ticker, length of ticker <= 5, excluded words,                     
                        if word.isupper() and len(word) <= 5 and word not in blacklist and word in us:
                            
                            # unique comments, try/except for key errors
                            if uniqueCmt and auth not in goodAuth:
                                try: 
                                    if auth in cmt_auth[word]: break
                                except: pass
                                
                            # counting tickers
                            if word in tickers:
                                tickers[word] += 1
                                a_comments[word].append(comment.body)
                                cmt_auth[word].append(auth)
                                count += 1
                            else:                               
                                tickers[word] = 1
                                cmt_auth[word] = [auth]
                                a_comments[word] = [comment.body]
                                count += 1    


# sorts the dictionary
symbols = dict(sorted(tickers.items(), key=lambda item: item[1], reverse = True))
top_picks = list(symbols.keys())[0:picks]
time = (time.time() - start_time)

# print top picks
print("It took {t:.2f} seconds to analyze {c} comments in {p} posts in {s} subreddits.\n".format(t=time, c=c_analyzed, p=posts, s=len(subs)))
print("Posts analyzed saved in titles")
#for i in titles: print(i)  # prints the title of the posts analyzed


print(f"\n{picks} most mentioned picks: ")
times = []
top = []
for i in top_picks:
    print(f"{i}: {symbols[i]}")
    times.append(symbols[i])
    top.append(f"{i}: {symbols[i]}")

It took 755.45 seconds to analyze 24295 comments in 101 posts in 4 subreddits.

Posts analyzed saved in titles

10 most mentioned picks: 
GME: 992
PLTR: 96
TSLA: 58
NIO: 50
APHA: 33
AAPL: 29
GE: 29
TLRY: 24
RKT: 24
BB: 22


In [5]:
symbols

{'GME': 992,
 'PLTR': 96,
 'TSLA': 58,
 'NIO': 50,
 'APHA': 33,
 'AAPL': 29,
 'GE': 29,
 'TLRY': 24,
 'RKT': 24,
 'BB': 22,
 'AMD': 21,
 'RIOT': 12,
 'E': 11,
 'TD': 10,
 'NOK': 9,
 'O': 9,
 'R': 9,
 'CRSR': 9,
 'DIS': 9,
 'NEW': 8,
 'MT': 8,
 'SPCE': 8,
 'MVIS': 8,
 'T': 8,
 'F': 8,
 'W': 7,
 'WMT': 7,
 'TSM': 7,
 'AN': 7,
 'FREE': 7,
 'MSM': 7,
 'GOEV': 6,
 'DKNG': 6,
 'H': 6,
 'AMZN': 6,
 'UNFI': 6,
 'HE': 6,
 'EAT': 6,
 'PH': 6,
 'G': 6,
 'HEAR': 6,
 'SQ': 5,
 'PLUG': 5,
 'L': 5,
 'AI': 5,
 'LMND': 5,
 'ABNB': 5,
 'X': 5,
 'V': 5,
 'CAT': 5,
 'BILL': 5,
 'NICE': 5,
 'RC': 5,
 'C': 5,
 'TA': 5,
 'TV': 5,
 'NVDA': 4,
 'MAN': 4,
 'OI': 4,
 'MSFT': 4,
 'BNGO': 4,
 'BA': 4,
 'CALM': 4,
 'TWO': 4,
 'REAL': 4,
 'FUND': 4,
 'PE': 4,
 'RUN': 3,
 'PUMP': 3,
 'WKHS': 3,
 'MARA': 3,
 'FUBO': 3,
 'DASH': 3,
 'TDOC': 3,
 'EVER': 3,
 'SNAP': 3,
 'DOW': 3,
 'JPM': 3,
 'ROCK': 3,
 'SI': 3,
 'PLAN': 3,
 'HA': 3,
 'PLAY': 3,
 'PSA': 3,
 'VERY': 3,
 'COST': 3,
 'ET': 3,
 'D': 3,
 'FT': 3,
 'KMB': 3,
 

In [6]:
a_comments

{'NVDA': ['NVDA I see you looking all cute stair stepping up',
  'NVDA, AAPL, VIAC, AMD, SBUX, MAR are all big winners for me.  Accumulating NVDA and AMD on Friday and Monday has paid off big.  I still maintain the NVDA and AMD are going back to prior highs.',
  'NVDA a buy right now or wait for a little downturn?',
  'Started investing in January... literally negative on every single investment. Plus I don’t think I picked many risky/speculative companies either!! I own $DIS $MTCH $SQ $MELI $NVDA Etc (mostly Large cap tech companies)...\nwhy has the market been sooo terrible to me?? any theories?'],
 'APHA': ['APHA GONNA RUN AH',
  'TLRY and APHA are parallel but they’re not in proportion to the share exchange',
  'APHA about to pop',
  'APHA quietly making me $$',
  "If you missed the short window to get APHA under $15 in the last week, for the love of God don't make the same mistake while it's still under $20.",
  'Only regret I have is not loading up more APHA at $16 and using that

In [7]:
# Applying Sentiment Analysis
scores, s = {}, {}
 
vader = SentimentIntensityAnalyzer()
# adding custom words from data.py 
vader.lexicon.update(new_words)

picks_sentiment = list(symbols.keys())[0:]
for symbol in picks_sentiment:
    stock_comments = a_comments[symbol]
    for cmnt in stock_comments:
        score = vader.polarity_scores(cmnt)
        if symbol in s:
            s[symbol][cmnt] = score
        else:
            s[symbol] = {cmnt:score}      
        if symbol in scores:
            for key, _ in score.items():
                scores[symbol][key] += score[key]
        else:
            scores[symbol] = score
            
    # calculating avg.
    for key in score:
        scores[symbol][key] = scores[symbol][key] / symbols[symbol]
        scores[symbol][key]  = "{pol:.3f}".format(pol=scores[symbol][key])

In [23]:
# printing sentiment analysis 
print(f"\nSentiment analysis of top picks:")
df = pd.DataFrame(scores)
df.index = ['Bearish', 'Neutral', 'Bullish', 'Total/Compound']
df = df.T
print(df)


Sentiment analysis of top picks:
     Bearish Neutral Bullish Total/Compound
GME    0.088   0.765   0.147          0.151
PLTR   0.068   0.765   0.167          0.245
TSLA   0.074   0.785   0.141          0.191
NIO    0.059   0.741   0.200          0.261
APHA   0.041   0.843   0.117          0.094
...      ...     ...     ...            ...
SNOW   0.000   0.674   0.326          0.700
UNH    0.066   0.895   0.039         -0.153
REV    0.082   0.798   0.119          0.699
SPWR   0.000   1.000   0.000          0.000
MAXN   0.080   0.655   0.265          0.911

[287 rows x 4 columns]


In [24]:
symb = pd.DataFrame([symbols]).T

In [25]:
df['Mentions'] = symb

In [26]:
df

,Bearish,Neutral,Bullish,Total/Compound,Mentions
GME,0.088,0.765,0.147,0.151,992
PLTR,0.068,0.765,0.167,0.245,96
TSLA,0.074,0.785,0.141,0.191,58
NIO,0.059,0.741,0.200,0.261,50
APHA,0.041,0.843,0.117,0.094,33
...,...,...,...,...,...
SNOW,0.000,0.674,0.326,0.700,1
UNH,0.066,0.895,0.039,-0.153,1
REV,0.082,0.798,0.119,0.699,1
SPWR,0.000,1.000,0.000,0.000,1


In [27]:
df.to_csv('tickers.csv')

In [28]:
df = pd.read_csv('tickers.csv')
df.rename(columns={'Unnamed: 0' : 'Tickers'}, inplace = True)

In [32]:
fig = px.treemap(df.iloc[:11], path=['Tickers'], values='Mentions')
fig.show()

In [35]:
a_comments['TSLA']

['TSLA is being so mean to me 😞',
 'TSLA is going to make a big move in one direction or the other. My calls say “up”. If we break $705 this week we’re hitting $750 soon.',
 'TSLA Put Credit Spreads have been good to me. Theta gang finally getting back on the W train',
 'TSLA puts to hedge in case of sour hour??',
 'TSLA 3/12 700C a good idea?',
 'TSLA what a shit way to start power hour',
 'need TSLA to gap up tomorrow plz. Anyone holding calls for 3/12?',
 'TSLA preparing another flight to Mars?',
 'TSLA POP POP POP PLEASE\n\nGo back over $700!!!',
 'All I ask is for TSLA to close above $700',
 'Wait until PLTR announces deal with TSLA',
 'TSLA will hit $1000 by EOY.',
 '10k open interest on 3/12 TSLA 700c 17k open interest on 3/19 TSLA 700c. If it pops 700 we’re in for a giant run when the MMS have to cover by next Tuesday. You know what to do. \n3/19 $TSLA 760c 🚀🚀🚀🚀',
 'Is TSLA fueling up the rocket?',
 'Hold TSLA calls that expire tomorrow or cut losses?',
 'Bought some TSLA at 69